In [183]:
#导入模块，导入数据，处理时间--train_log
import numpy as np
import pandas as pd
from pandas import DataFrame
from collections import Counter
train_log = pd.read_csv('train/train_log.csv',sep='\t',parse_dates = ['OCC_TIM'])#parse_dates:这是指定含有时间数据信息的列。
train_log['hour'] = train_log.OCC_TIM.map(lambda x:x.hour)
train_log['day'] = train_log.OCC_TIM.map(lambda x:x.day)
train_log.head(10)#显示excel表格

,USRID,EVT_LBL,OCC_TIM,TCH_TYP,hour,day
0,10002,163-577-913,2018-03-22 16:31:44,0,16,22
1,10002,163-578-914,2018-03-22 16:31:18,0,16,22
2,10002,259-924-1525,2018-03-22 16:31:15,0,16,22
3,10002,326-1040-1677,2018-03-06 12:08:51,0,12,6
4,10002,326-1041-1678,2018-03-09 14:40:22,0,14,9
5,10002,326-1041-1678,2018-03-16 17:45:42,0,17,16
6,10002,359-1234-2004,2018-03-15 10:40:35,0,10,15
7,10002,520-1836-3640,2018-03-06 12:07:02,0,12,6
8,10002,520-1836-3640,2018-03-09 11:23:01,0,11,9
9,10002,520-1836-3640,2018-03-22 16:31:13,0,16,22


In [184]:
#将train_log 中  EVT_LB--拆分成了三个模块加在后面--保存成df3
df2=train_log["EVT_LBL"].str.split('-', expand=True)#给EVT_LBL字段分开成三个，expand=True参数将字符串拆分成多列，返回一个数据框
df3=train_log.drop('EVT_LBL', axis=1).join(df2)#原表格按照列方向，删除EVT_LBL，然后将后来的新的一列表格df2水平连接
#df3=train_log.join(df2)#暂时不删除EVT_LBL
df3.rename(columns={0: 'EVT_LBL_1', 1: 'EVT_LBL_2', 2: 'EVT_LBL_3'}, inplace=True)#将上面列名称0，1，2改成新的名称
df3.head(10)

,USRID,OCC_TIM,TCH_TYP,hour,day,EVT_LBL_1,EVT_LBL_2,EVT_LBL_3
0,10002,2018-03-22 16:31:44,0,16,22,163,577,913
1,10002,2018-03-22 16:31:18,0,16,22,163,578,914
2,10002,2018-03-22 16:31:15,0,16,22,259,924,1525
3,10002,2018-03-06 12:08:51,0,12,6,326,1040,1677
4,10002,2018-03-09 14:40:22,0,14,9,326,1041,1678
5,10002,2018-03-16 17:45:42,0,17,16,326,1041,1678
6,10002,2018-03-15 10:40:35,0,10,15,359,1234,2004
7,10002,2018-03-06 12:07:02,0,12,6,520,1836,3640
8,10002,2018-03-09 11:23:01,0,11,9,520,1836,3640
9,10002,2018-03-22 16:31:13,0,16,22,520,1836,3640


# 下面是具体对df3操作的方法

In [185]:
# 得到每个用户在模块1-2-3中的最常用的模块（多个频繁模块则取平均）
df3['module_1']=np.array(df3['EVT_LBL_1'],dtype='int64')
df3['module_2']=np.array(df3['EVT_LBL_2'],dtype='int64')
df3['module_3']=np.array(df3['EVT_LBL_3'],dtype='int64')
EVT_LBL_1_module=df3.groupby('USRID', as_index = False)['module_1'].agg({'EVT_LBL_1_module':lambda x: np.mean(pd.Series.mode(x))})
EVT_LBL_2_module=df3.groupby('USRID', as_index = False)['module_2'].agg({'EVT_LBL_2_module':lambda x: np.mean(pd.Series.mode(x))})
EVT_LBL_3_module=df3.groupby('USRID', as_index = False)['module_3'].agg({'EVT_LBL_3_module':lambda x: np.mean(pd.Series.mode(x))})

In [189]:
EVT_LBL_3_module.isnull().any()

USRID               False
EVT_LBL_3_module    False
dtype: bool

In [ ]:
#value_counts 自动排序，取第一个就是最频繁的里面最大的
df3.groupby('USRID')['EVT_LBL_1'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [ ]:
#调用scipy 中的办法
from scipy import stats
df3.groupby('USRID')['EVT_LBL_1'].agg(lambda x: stats.mode(x)[0][0]).reset_index()

In [ ]:
# 调用Counter.most_common方法--写函数取出DF中 EVT_LBL_1 列 中的出现次数最多的元素，转化成矩阵数值 返回
def trans(dd):
    kk=[word for word, word_count in Counter(dd['EVT_LBL_1']).most_common(1)]
    numpy_df=np.array(kk,dtype='int64')
    return numpy_df
EVT_LBL_1_module = df3.groupby(by= ['USRID'], as_index = False)['EVT_LBL_1'].agg({'EVT_LBL_1_module':lambda x:trans(dd)})
EVT_LBL_1_module

# 下面是摸索出来的几个不同的求频率最高的元素的值的方法

In [ ]:
#1----------Counter.most_common方法，找到频率最高的值
from collections import Counter
word_counts = Counter(df3.head(10)['EVT_LBL_1'])
print(word_counts.most_common(3))#一对数值

most_common_words=[word for word, word_count in Counter(df3.head(10)['EVT_LBL_1']).most_common(1)]
print (most_common_words)

from pandas import DataFrame
most_common_words=DataFrame([word for word, word_count in Counter(df3['EVT_LBL_1']).most_common(1)])
print (most_common_words)

In [ ]:
#2----------df转化为numpy,再求众数np.argmax(np.bincount)
example=df3['EVT_LBL_1']
numpy_df=np.array(example,dtype='int64')
print(numpy_df)
numpy_df.dtype
np.argmax(np.bincount(numpy_df))
#np.argmax(np.bincount(['EVT_LBL_1']))

In [ ]:
#3------------mode 方法求众数
dd=df3[:20]
kk=DataFrame(dd['EVT_LBL_1'].dropna().mode().values)
kk